In [0]:
from google.colab import auth
auth.authenticate_user()

In [0]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!4/AADDHkzfY0tqlcDQEFzLP0yU--Zo4L0fI6i7rrYLz5I8jszA8btkeb8apt-get -y install -qq google-drive-ocamlfuse fuse
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

Preconfiguring packages ...
Selecting previously unselected package cron.
(Reading database ... 18298 files and directories currently installed.)
Preparing to unpack .../00-cron_3.0pl1-128ubuntu5_amd64.deb ...
Unpacking cron (3.0pl1-128ubuntu5) ...
Selecting previously unselected package libapparmor1:amd64.
Preparing to unpack .../01-libapparmor1_2.11.0-2ubuntu17.1_amd64.deb ...
Unpacking libapparmor1:amd64 (2.11.0-2ubuntu17.1) ...
Selecting previously unselected package libdbus-1-3:amd64.
Preparing to unpack .../02-libdbus-1-3_1.10.22-1ubuntu1_amd64.deb ...
Unpacking libdbus-1-3:amd64 (1.10.22-1ubuntu1) ...
Selecting previously unselected package dbus.
Preparing to unpack .../03-dbus_1.10.22-1ubuntu1_amd64.deb ...
Unpacking dbus (1.10.22-1ubuntu1) ...
Selecting previously unselected package dirmngr.
Preparing to unpack .../04-dirmngr_2.1.15-1ubuntu8_amd64.deb ...
Unpacking dirmngr (2.1.15-1ubuntu8) ...
Selecting previously unselected package distro-info-data.
Preparing to unpack .../0

invoke-rc.d: could not determine current runlevel
invoke-rc.d: policy-rc.d denied execution of start.
Setting up dirmngr (2.1.15-1ubuntu8) ...
Setting up cron (3.0pl1-128ubuntu5) ...
Adding group `crontab' (GID 102) ...
Done.
update-rc.d: warning: start and stop actions are no longer supported; falling back to defaults
update-rc.d: warning: stop runlevel arguments (1) do not match cron Default-Stop values (none)
invoke-rc.d: could not determine current runlevel
invoke-rc.d: policy-rc.d denied execution of start.
Setting up libdbus-1-3:amd64 (1.10.22-1ubuntu1) ...
Setting up kmod (24-1ubuntu2) ...
Setting up libdbus-glib-1-2:amd64 (0.108-2) ...
Setting up python3-gi (3.24.1-2build1) ...
Setting up module-init-tools (24-1ubuntu2) ...
Setting up python3-software-properties (0.96.24.17) ...
Setting up dbus (1.10.22-1ubuntu1) ...
Setting up python-apt (1.4.0~beta3build2) ...
Setting up python3-dbus (1.2.4-1build3) ...
Setting up python-software-properties (0.96.24.17) ...
Setting up softwar

In [2]:
!mkdir -p my_drive
!google-drive-ocamlfuse my_drive

fuse: mountpoint is not empty
fuse: if you are sure this is safe, use the 'nonempty' mount option


In [0]:
#!mkdir -p my_drive/ML_Project

In [0]:
#!ls -p my_drive/ML_Project

In [3]:
import numpy
import pandas
from keras.models import Sequential
from keras.layers import Dense, Dropout, BatchNormalization
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score, KFold, GridSearchCV
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.preprocessing.data import QuantileTransformer
from sklearn.pipeline import Pipeline
from scipy import stats

Using TensorFlow backend.


In [15]:
!ls -p my_drive/ML_Project/X_train.txt

my_drive/ML_Project/X_train.txt


In [0]:
import os
base = 'my_drive'+os.sep+'ML_Project'+os.sep
X = pandas.read_csv(base+"X_train.txt", delim_whitespace=True, header=None)
#X = X.sample(n=10000,replace="False")
Y = pandas.read_csv(base+"Y_train.txt", delim_whitespace=True, header=None)
#Y = Y.sample(n=10000,replace="False")
dimensions = 14

In [0]:
Y.columns.values[0] = '14'
df_new = pandas.concat([X, Y],axis=1)
df_new.drop_duplicates(inplace=True)
df_new = df_new[(numpy.abs(stats.zscore(df_new)) < 3).all(axis=1)]

X = df_new[list(df_new.columns[0:14])]
Y = df_new.iloc[:,-1]

In [0]:
dimensions = 11
X.drop(X.columns[[5, 6, 12]], axis=1, inplace=True)

In [0]:
X = X.values
Y = Y.values

In [129]:
X.shape

(100000, 11)

In [0]:
def baseline_model(opti='adam',init='uniform'):
  model = Sequential()
#  model.add(Dense(2**6, input_dim=14, kernel_initializer=init, kernel_regularizer = 'l2', activation='relu'))
#  model.add(Dense(2**5, kernel_initializer=init, kernel_regularizer = 'l2', activation='relu'))
  model.add(Dense(20, input_dim=dimensions, kernel_initializer=init, kernel_regularizer = 'l2', activation='relu'))
  model.add(Dense(12, kernel_initializer=init, kernel_regularizer = 'l2', activation='relu'))
  model.add(Dense(1, kernel_initializer=init, kernel_regularizer = 'l2', activation='linear'))
  model.compile(loss='mse', optimizer=opti, metrics=['accuracy'])
  return model

In [0]:
# fix random seed for reproducibility
seed = 7
numpy.random.seed(seed)
# evaluate model with standardized dataset
# estimator = KerasRegressor(build_fn=baseline_model, epochs=50, batch_size=5, verbose=1)

In [0]:
# evaluate model with standardized dataset
estimators = []
estimators.append(('standardize', StandardScaler()))
#estimator = KerasRegressor(build_fn=baseline_model, opti='adam', init='normal', epochs=20, batch_size=50, verbose=0)
estimator = KerasRegressor(build_fn=baseline_model, epochs=20, batch_size=100, verbose=0)
estimators.append(('mlp', estimator))
pipeline = Pipeline(estimators)

In [133]:
kfold = KFold(n_splits=10, random_state=seed)
results = cross_val_score(pipeline, X, Y, cv=kfold)
print("Standardized: %.2f (%.2f) MSE" % (results.mean(), results.std()))

Standardized: -3.55 (0.45) MSE


In [120]:
#results_ = results
results.mean()

-0.5816243524355047

In [20]:
#dropout_rate = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
init = ['glorot_uniform', 'normal', 'uniform']
epochs = [50]
batches = [5,10]
#weight_constraint = [1, 2, 3, 4, 5]
param_grid = dict(
#    opti=optimizers, 
#  dropout_rate=dropout_rate, 
  epochs=epochs,
  batch_size=batches, 
#  weight_constraint=weight_constraint, 
#  init=init
)
grid_2 = GridSearchCV(estimator=estimator, cv=5, param_grid=param_grid, verbose=2)
grid_result_2 = grid_2.fit(X, Y)

Fitting 5 folds for each of 2 candidates, totalling 10 fits
[CV] batch_size=5, epochs=50 .........................................
[CV] .......................... batch_size=5, epochs=50, total= 2.0min
[CV] batch_size=5, epochs=50 .........................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  2.1min remaining:    0.0s


[CV] .......................... batch_size=5, epochs=50, total= 2.1min
[CV] batch_size=5, epochs=50 .........................................
[CV] .......................... batch_size=5, epochs=50, total= 2.1min
[CV] batch_size=5, epochs=50 .........................................
[CV] .......................... batch_size=5, epochs=50, total= 2.1min
[CV] batch_size=5, epochs=50 .........................................
[CV] .......................... batch_size=5, epochs=50, total= 2.1min
[CV] batch_size=10, epochs=50 ........................................
[CV] ......................... batch_size=10, epochs=50, total= 1.1min
[CV] batch_size=10, epochs=50 ........................................
[CV] ......................... batch_size=10, epochs=50, total= 1.1min
[CV] batch_size=10, epochs=50 ........................................
[CV] ......................... batch_size=10, epochs=50, total= 1.1min
[CV] batch_size=10, epochs=50 ........................................
[CV] .

[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed: 16.0min finished


In [17]:
# summarize results
print ('Best: %f using %s' % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
  print ('%f (%f) with: %r' % (mean, stdev, param))

NameError: ignored

In [21]:
# summarize results
print ('Best: %f using %s' % (grid_result_2.best_score_, grid_result_2.best_params_))
means = grid_result_2.cv_results_['mean_test_score']
stds = grid_result_2.cv_results_['std_test_score']
params = grid_result_2.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
  print ('%f (%f) with: %r' % (mean, stdev, param))

Best: -4.363742 using {'batch_size': 5, 'epochs': 50}
-4.363742 (0.860694) with: {'batch_size': 5, 'epochs': 50}
-4.364276 (0.860905) with: {'batch_size': 10, 'epochs': 50}


In [0]:
X_test = pandas.read_csv(base+"X_test.txt", delim_whitespace=True, header=None)

In [0]:
X_test.drop(X_test.columns[[5, 6, 12]], axis=1, inplace=True)

In [0]:
# evaluate model with standardized dataset
X_test = X_test.values
pipeline.fit(X,Y)
prediction = pipeline.predict(X_test)

In [0]:
numpy.std(prediction)
import copy
predictionX = copy.deepcopy(prediction)

In [0]:
fh = open(base+'predictions.csv','w')
fh.write('ID,Prediction\n')
for i,yi in enumerate(prediction):
    fh.write('{},{}\n'.format(i,yi))
fh.close()

In [122]:
unique, counts = numpy.unique(prediction, return_counts=True)

print (numpy.asarray((unique, counts)).T)

[[0.17075202 1.        ]
 [0.17075348 2.        ]
 [0.17075382 1.        ]
 ...
 [3.11205339 1.        ]
 [3.128335   1.        ]
 [3.31371856 1.        ]]


In [147]:
numpy.histogram(predictionX)

(array([84673, 12828,  2010,   302,    93,    55,    18,    12,     6,
            3]),
 array([0.17075202, 0.48504867, 0.79934533, 1.11364198, 1.42793863,
        1.74223529, 2.05653194, 2.3708286 , 2.68512525, 2.9994219 ,
        3.31371856]))

In [1]:
numpy.histogram(prediction)

NameError: ignored

In [0]:
abc = pandas.DataFrame(prediction)
#abc[0].value_counts()

In [161]:
def final_model():
  model = Sequential()
  model.add(Dense(30, input_dim=14, kernel_initializer='normal', activation='relu'))
  model.add(Dense(5, kernel_initializer='normal', activation='relu'))
  model.add(Dense(1, kernel_initializer='normal'))
  model.compile(loss='mean_squared_error', optimizer='Nadam')
  return model
estimators = []
#estimators.append(('standardize', StandardScaler()))
#estimators.append(('standardize', RobustScaler(quantile_range=(25, 75))))
#estimators.append(('standardize', QuantileTransformer(output_distribution='uniform')))
estimators.append(('standardize', QuantileTransformer(output_distribution='normal')))
estimator = KerasRegressor(build_fn=baseline_model, epochs=20, batch_size=100, verbose=0)
estimators.append(('mlp', estimator))
pipeline = Pipeline(estimators)
kfold = KFold(n_splits=10, random_state=seed)
results = cross_val_score(pipeline, X, Y, cv=kfold)
print("Standardized: %.2f (%.2f) MSE" % (results.mean(), results.std()))

Standardized: -3.54 (0.45) MSE


In [0]:
robust = pipeline.fit(X,Y)

In [0]:
# evaluate model with standardized dataset
X_test = X_test.values
pipeline.fit(X,Y)
prediction = pipeline.predict(X_test)
fh = open(base+'predictions.csv','w')
fh.write('ID,Prediction\n')
for i,yi in enumerate(prediction):
    fh.write('{},{}\n'.format(i,yi))
fh.close()